In [11]:
import csv
from functools import reduce

In [2]:
class User: # пользователь
    name = ''
    movie_rates = []
    avg_rate = 0

In [3]:
class Movie_rate:  # фильм
    def __init__(self, name, rate):
        self.name = name
        self.rate = rate

In [4]:
def parse_movie_names(row):
    row = list(map(lambda name: name.strip(), row))
    return list(filter(lambda name: name != '', row))

In [5]:
def load_users(filename):
    data_file = open(filename)
    reader = csv.reader(data_file)
    
    movie_names = parse_movie_names(next(reader))
    users = []
    
    for row in reader:
        user = User()
        user.name = row[0]
        user.movie_rates = list(
            map(lambda movie_name, movie_rate: Movie_rate(movie_name, int(movie_rate)), movie_names, row[1:])
        )
        
        users.append(user)
        
    return users

In [17]:
def calculate_avg_rate(user): # считаем среднее
    valid_rates = list(filter(lambda movie_rate: movie_rate.rate != -1, user.movie_rates))
    return reduce(lambda summ, movie_rate: summ + movie_rate.rate, valid_rates, 0) / len(valid_rates)

In [27]:
users = load_users('./data.csv')
for user in users:
    print(user.name)
    print(calculate_avg_rate(user))
    summ = 0
    d = 0
    for movie_rate in user.movie_rates:
        if movie_rate.rate != -1:
            summ += movie_rate.rate
            d += 1
    print(summ / d)

User 1
2.9523809523809526
2.9523809523809526
User 2
3.2916666666666665
3.2916666666666665
User 3
2.8
2.8
User 4
3.142857142857143
3.142857142857143
User 5
3.3076923076923075
3.3076923076923075
User 6
3.16
3.16
User 7
2.88
2.88
User 8
3.2916666666666665
3.2916666666666665
User 9
2.8076923076923075
2.8076923076923075
User 10
2.72
2.72
User 11
3.125
3.125
User 12
3.8461538461538463
3.8461538461538463
User 13
3.1538461538461537
3.1538461538461537
User 14
3.0
3.0
User 15
3.375
3.375
User 16
3.4074074074074074
3.4074074074074074
User 17
3.0
3.0
User 18
2.7037037037037037
2.7037037037037037
User 19
3.125
3.125
User 20
3.1818181818181817
3.1818181818181817
User 21
2.96
2.96
User 22
3.217391304347826
3.217391304347826
User 23
3.391304347826087
3.391304347826087
User 24
2.814814814814815
2.814814814814815
User 25
2.76
2.76
User 26
2.92
2.92
User 27
3.3333333333333335
3.3333333333333335
User 28
2.923076923076923
2.923076923076923
User 29
3.2
3.2
User 30
2.9642857142857144
2.9642857142857144
User 